# **Dataset:** *Digits-MNIST*
# **Generaciones:** *20 generaciones*
# **Porcentaje de cruza:** *70 %*
# **Porcentaje de mutación:** *70 %*
Ejecutado en Google Colaboratory

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 3472641183540115758
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 16154099712
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 10983097666024874074
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
 xla_global_id: 416903419]

In [4]:
!cp /content/drive/MyDrive/DGA_digits/DataReader.py /content
!cp /content/drive/MyDrive/DGA_digits/Decoding.py /content
!cp /content/drive/MyDrive/DGA_digits/DistributedTraining.py /content
!cp /content/drive/MyDrive/DGA_digits/EncodingClass.py /content
!cp /content/drive/MyDrive/DGA_digits/Operators.py /content
# Ruta del dataset
!cp /content/drive/MyDrive/Datasets/digits.zip /content

In [ ]:
# Descomprimir dataset
!unzip digits.zip

# Ejecución 1

In [6]:
## MÓDULOS ##########################################################################################
from Operators import *
from EncodingClass import *
from Decoding import *
from DataReader import *
from DistributedTraining import *

## LIBRERÍAS ########################################################################################
import numpy as np
from torch import optim
import pandas as pd
import timeit
import torch
from torch import nn
import pickle

## HIPERPARÁMETROS DE LA CNN ########################################################################
loss_func = nn.NLLLoss(reduction = "sum") #Defining loss function
lr = 1e-4 #Defining learning rate
#Maximun and minimum numbers of layers to initialize networks
min_conv = 2
max_conv = 5
min_full = 1
max_full = 4
w = 0.3 #penalization weight
max_params = 2e6
num_epochs = 10

## PARÁMETROS GA ####################################################################################
N = 20 #Population size
cr = 0.7 #Crossover rate
mr = 0.7 #Mutation rate
#T = 50 #Number of generations
t_size = 5 #tournament size
train_dl, test_dl = loading_data()
#device1 = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device1 = torch.device("cuda:0")
device1 = torch.device("cuda")

## CREAR POBLACIÓN ##################################################################################
pop = []
bestAcc = []
bestF = []
bestParams = []
print("Iniciando población...\n")
while len(pop) < N:
  while True:
    try:
      acc_list = []
      #Creating genomes (genetic encoding)
      e1 = Encoding(min_conv,max_conv,min_full,max_full)
      #Decoding the network
      network1 = decoding(e1)
      #Creating the CNN
      cnn1 = CNN(e1, network1[0], network1[1], network1[2])
      print("Individuo {} creado:".format(len(pop)+1))
      #Training the CNN
      training('1', device1, cnn1, num_epochs, loss_func,train_dl, test_dl, lr, w, max_params, acc_list)
      #Add individual to population
      pop.append([e1,acc_list[0][1],acc_list[0][2],acc_list[0][3]])
      print("Individuo {} evaluado.\n".format(len(pop)))
      break
    except:
      pass

print("Población creada!!")

Iniciando población...

Individuo 1 creado:
Individuo 1 evaluado.

Individuo 2 creado:
Individuo 2 evaluado.

Individuo 3 creado:
Individuo 3 evaluado.

Individuo 4 creado:
Individuo 4 evaluado.

Individuo 5 creado:
Individuo 5 evaluado.

Individuo 6 creado:
Individuo 6 evaluado.

Individuo 7 creado:
Individuo 7 evaluado.

Individuo 8 creado:
Individuo 8 evaluado.

Individuo 9 creado:
Individuo 9 evaluado.

Individuo 10 creado:
Individuo 10 evaluado.

Individuo 11 creado:
Individuo 11 evaluado.

Individuo 12 creado:
Individuo 12 evaluado.

Individuo 13 creado:
Individuo 13 evaluado.

Individuo 14 creado:
Individuo 14 evaluado.

Individuo 15 creado:
Individuo 15 evaluado.

Individuo 16 creado:
Individuo 16 evaluado.

Individuo 17 creado:
Individuo 17 creado:


/usr/local/lib/python3.7/dist-packages/torch/nn/init.py:403: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Individuo 17 evaluado.

Individuo 18 creado:
Individuo 18 evaluado.

Individuo 19 creado:
Individuo 19 evaluado.

Individuo 20 creado:
Individuo 20 creado:
Individuo 20 evaluado.

Población creada!!


In [7]:
#Mostrar aptitudes, precisión y num. parámetros de población inicial
for network in pop:
  print("f:{}, acc:{}, param:{}".format(network[1],network[2],network[3]))
max(pop, key = lambda x: x[1])

f:0.9615424666666667, acc:0.9610952380952381, param:74828
f:0.8510971999999999, acc:0.7884285714285715, param:5352
f:0.9630706333333333, acc:0.9850476190476191, param:176418
f:0.7461987999999999, acc:0.6387142857142857, param:6008
f:0.9457601666666666, acc:0.9822380952380952, param:278710
f:0.9776842666666665, acc:0.9773809523809524, param:43216
f:0.8193918666666666, acc:0.7423809523809524, param:1832
f:0.9650712666666665, acc:0.9570952380952381, param:32636
f:0.8877636, acc:0.8421428571428572, param:11576
f:0.9669494666666667, acc:0.9565238095238096, param:17448
f:0.9597140666666666, acc:0.9458095238095238, param:15684
f:0.9783872, acc:0.9718571428571429, param:12752
f:0.9513837, acc:0.9347142857142857, param:19442
f:0.9460074, acc:0.9364285714285714, param:63284
f:0.8664312333333333, acc:0.8109047619047619, param:8014
f:0.9613474666666666, acc:0.9460952380952381, param:6128
f:0.9327265666666666, acc:0.904952380952381, param:4934
f:0.9562768333333334, acc:0.9404761904761905, param:137

 0.9783872,
 0.9718571428571429,
 12752]

In [8]:
T = 10 #Number of generations
print("Iniciando genético...")
for t in range(T):
  print('Generation: ', t)
    
  #Parents Selection
  parents = []
  while len(parents) < int(N/2):
    #Tournament Selection
    tournament = random.sample(pop, t_size)
    p1 = selection(tournament, 'max')
    tournament = random.sample(pop, t_size)
    p2 = selection(tournament, 'max')
    while p1 == p2:
      tournament = random.sample(pop, t_size)
      p2 = selection(tournament, 'max')  
        
    parents.append(p1)
    parents.append(p2)
    
    #Reproduction
  offspring = []
  while len(offspring) < int(N/2):
    par = random.sample(parents, 2)
    #Crossover + Mutation
    if cr >= random.uniform(0,1): #Crossover
      p1 = par[0][0]
      p2 = par[1][0]
      c1, c2 = crossover(p1, p2)
            
    #Mutation
    if mr >= random.uniform(0,1):
      mutation(c1)
            
    if mr >= random.uniform(0,1):
      mutation(c2)

    #Try to evaluate the first offspring
    try:
      network1 = decoding(c1)
      cnn1 = CNN(c1, network1[0], network1[1], network1[2])
      acc1_list = []
      training('1', device1, cnn1, num_epochs, loss_func,train_dl, test_dl, lr, w, max_params, acc1_list)
      print("Descendiente {} evaluado.".format(len(offspring)+1))
      offspring.append([c1,acc1_list[0][1],acc1_list[0][2],acc1_list[0][3]])
    except:
      c1_random = random.sample(pop,1)[0]
      print("Descendiente {} aleatorio de pop.".format(len(offspring)+1))
      offspring.append([c1_random[0],c1_random[1],c1_random[2],c1_random[3]])

    #Try to evaluate the second offspring
    try:
      network2 = decoding(c2)
      cnn2 = CNN(c2, network2[0], network2[1], network2[2])
      acc2_list = []
      training('2', device1, cnn2, num_epochs, loss_func,train_dl, test_dl, lr, w, max_params, acc2_list)
      print("Descendiente {} evaluado.".format(len(offspring)+1))
      offspring.append([c2,acc2_list[0][1],acc2_list[0][2],acc2_list[0][3]])
    except:
      c2_random = random.sample(pop,1)[0]
      print("Descendiente {} aleatorio de pop.".format(len(offspring)+1))
      offspring.append([c2_random[0],c2_random[1],c2_random[2],c2_random[3]])

  #Replacement with elitism
  pop = pop + offspring
  pop.sort(reverse = True, key = lambda x: x[1])
  pop = pop[:N]
    
  leader = max(pop, key = lambda x: x[1])
  bestAcc.append(leader[2])
  bestF.append(leader[1])
  bestParams.append(leader[3])
    
        
  print('Best fitness: ', leader[1])
  print('Best accuracy: ', leader[2])
  print('Best No. of Params: ', leader[3])
  print('No. of Conv. Layers: ', leader[0].n_conv)
  print('No. of FC Layers: ', leader[0].n_full)
  print('--------------------------------------------')

print("Fin genético!!")

Iniciando genético...
Generation:  0
Descendiente 1 evaluado.
Descendiente 2 evaluado.
Descendiente 3 evaluado.
Descendiente 4 evaluado.
Descendiente 5 evaluado.
Descendiente 6 evaluado.
Descendiente 7 evaluado.
Descendiente 8 evaluado.
Descendiente 9 evaluado.
Descendiente 10 evaluado.
Best fitness:  0.9783872
Best accuracy:  0.9718571428571429
Best No. of Params:  12752
No. of Conv. Layers:  4
No. of FC Layers:  1
--------------------------------------------
Generation:  1
Descendiente 1 evaluado.
Descendiente 2 aleatorio de pop.


/usr/local/lib/python3.7/dist-packages/torch/nn/init.py:403: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Descendiente 3 evaluado.
Descendiente 4 evaluado.
Descendiente 5 evaluado.
Descendiente 6 evaluado.
Descendiente 7 evaluado.
Descendiente 8 evaluado.
Descendiente 9 evaluado.
Descendiente 10 evaluado.
Best fitness:  0.9849310333333332
Best accuracy:  0.9846190476190476
Best No. of Params:  28682
No. of Conv. Layers:  4
No. of FC Layers:  1
--------------------------------------------
Generation:  2
Descendiente 1 evaluado.
Descendiente 2 evaluado.
Descendiente 3 evaluado.
Descendiente 4 evaluado.
Descendiente 5 evaluado.
Descendiente 6 evaluado.
Descendiente 7 evaluado.
Descendiente 8 evaluado.
Descendiente 9 evaluado.
Descendiente 10 evaluado.
Best fitness:  0.9849310333333332
Best accuracy:  0.9846190476190476
Best No. of Params:  28682
No. of Conv. Layers:  4
No. of FC Layers:  1
--------------------------------------------
Generation:  3
Descendiente 1 evaluado.
Descendiente 2 evaluado.
Descendiente 3 evaluado.
Descendiente 4 evaluado.
Descendiente 5 evaluado.
Descendiente 6 evalua

In [9]:
T = 10 #Number of generations
print("Iniciando genético...")
for t in range(T):
  print('Generation: ', t)
    
  #Parents Selection
  parents = []
  while len(parents) < int(N/2):
    #Tournament Selection
    tournament = random.sample(pop, t_size)
    p1 = selection(tournament, 'max')
    tournament = random.sample(pop, t_size)
    p2 = selection(tournament, 'max')
    while p1 == p2:
      tournament = random.sample(pop, t_size)
      p2 = selection(tournament, 'max')  
        
    parents.append(p1)
    parents.append(p2)
    
    #Reproduction
  offspring = []
  while len(offspring) < int(N/2):
    par = random.sample(parents, 2)
    #Crossover + Mutation
    if cr >= random.uniform(0,1): #Crossover
      p1 = par[0][0]
      p2 = par[1][0]
      c1, c2 = crossover(p1, p2)
            
    #Mutation
    if mr >= random.uniform(0,1):
      mutation(c1)
            
    if mr >= random.uniform(0,1):
      mutation(c2)

    #Try to evaluate the first offspring
    try:
      network1 = decoding(c1)
      cnn1 = CNN(c1, network1[0], network1[1], network1[2])
      acc1_list = []
      training('1', device1, cnn1, num_epochs, loss_func,train_dl, test_dl, lr, w, max_params, acc1_list)
      print("Descendiente {} evaluado.".format(len(offspring)+1))
      offspring.append([c1,acc1_list[0][1],acc1_list[0][2],acc1_list[0][3]])
    except:
      c1_random = random.sample(pop,1)[0]
      print("Descendiente {} aleatorio de pop.".format(len(offspring)+1))
      offspring.append([c1_random[0],c1_random[1],c1_random[2],c1_random[3]])

    #Try to evaluate the second offspring
    try:
      network2 = decoding(c2)
      cnn2 = CNN(c2, network2[0], network2[1], network2[2])
      acc2_list = []
      training('2', device1, cnn2, num_epochs, loss_func,train_dl, test_dl, lr, w, max_params, acc2_list)
      print("Descendiente {} evaluado.".format(len(offspring)+1))
      offspring.append([c2,acc2_list[0][1],acc2_list[0][2],acc2_list[0][3]])
    except:
      c2_random = random.sample(pop,1)[0]
      print("Descendiente {} aleatorio de pop.".format(len(offspring)+1))
      offspring.append([c2_random[0],c2_random[1],c2_random[2],c2_random[3]])

  #Replacement with elitism
  pop = pop + offspring
  pop.sort(reverse = True, key = lambda x: x[1])
  pop = pop[:N]
    
  leader = max(pop, key = lambda x: x[1])
  bestAcc.append(leader[2])
  bestF.append(leader[1])
  bestParams.append(leader[3])
    
        
  print('Best fitness: ', leader[1])
  print('Best accuracy: ', leader[2])
  print('Best No. of Params: ', leader[3])
  print('No. of Conv. Layers: ', leader[0].n_conv)
  print('No. of FC Layers: ', leader[0].n_full)
  print('--------------------------------------------')

print("Fin genético!!")

Iniciando genético...
Generation:  0
Descendiente 1 evaluado.
Descendiente 2 evaluado.
Descendiente 3 evaluado.
Descendiente 4 evaluado.
Descendiente 5 evaluado.
Descendiente 6 evaluado.
Descendiente 7 evaluado.
Descendiente 8 evaluado.
Descendiente 9 evaluado.
Descendiente 10 evaluado.
Best fitness:  0.9878475666666666
Best accuracy:  0.9892380952380952
Best No. of Params:  30794
No. of Conv. Layers:  5
No. of FC Layers:  1
--------------------------------------------
Generation:  1
Descendiente 1 evaluado.
Descendiente 2 evaluado.
Descendiente 3 evaluado.
Descendiente 4 evaluado.
Descendiente 5 evaluado.
Descendiente 6 evaluado.
Descendiente 7 evaluado.
Descendiente 8 evaluado.
Descendiente 9 evaluado.
Descendiente 10 evaluado.
Best fitness:  0.9878475666666666
Best accuracy:  0.9892380952380952
Best No. of Params:  30794
No. of Conv. Layers:  5
No. of FC Layers:  1
--------------------------------------------
Generation:  2
Descendiente 1 evaluado.
Descendiente 2 evaluado.
Descendie

/usr/local/lib/python3.7/dist-packages/torch/nn/init.py:403: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Descendiente 10 evaluado.
Best fitness:  0.9878475666666666
Best accuracy:  0.9892380952380952
Best No. of Params:  30794
No. of Conv. Layers:  5
No. of FC Layers:  1
--------------------------------------------
Generation:  3
Descendiente 1 aleatorio de pop.
Descendiente 2 evaluado.
Descendiente 3 evaluado.
Descendiente 4 evaluado.
Descendiente 5 evaluado.
Descendiente 6 evaluado.
Descendiente 7 evaluado.
Descendiente 8 evaluado.
Descendiente 9 evaluado.
Descendiente 10 aleatorio de pop.
Best fitness:  0.9878475666666666
Best accuracy:  0.9892380952380952
Best No. of Params:  30794
No. of Conv. Layers:  5
No. of FC Layers:  1
--------------------------------------------
Generation:  4
Descendiente 1 evaluado.
Descendiente 2 evaluado.
Descendiente 3 evaluado.
Descendiente 4 evaluado.
Descendiente 5 evaluado.
Descendiente 6 evaluado.
Descendiente 7 aleatorio de pop.
Descendiente 8 evaluado.
Descendiente 9 evaluado.
Descendiente 10 evaluado.
Best fitness:  0.9878475666666666
Best accurac

In [10]:
#Mostrar aptitudes, precisión y num. parámetros de población final
for network in pop:
  print("f:{}, acc:{}, param:{}".format(network[1],network[2],network[3]))
max(pop, key = lambda x: x[1])

f:0.9878475666666666, acc:0.9892380952380952, param:30794
f:0.9878475666666666, acc:0.9892380952380952, param:30794
f:0.9877514666666667, acc:0.9899523809523809, param:34768
f:0.9876273, acc:0.9891428571428571, param:31818
f:0.9876146666666665, acc:0.9895238095238095, param:33680
f:0.9874257, acc:0.9895714285714285, param:35162
f:0.9874257, acc:0.9895714285714285, param:35162
f:0.9872527999999999, acc:0.9885714285714285, param:31648
f:0.9872142333333334, acc:0.9883333333333333, param:30794
f:0.9871273, acc:0.9884285714285714, param:31818
f:0.9869606333333334, acc:0.9881904761904762, param:31818
f:0.9869475666666665, acc:0.9879523809523809, param:30794
f:0.9869142333333334, acc:0.987904761904762, param:30794
f:0.9868809000000001, acc:0.9878571428571429, param:30794
f:0.9866806333333333, acc:0.9880476190476191, param:33018
f:0.9865606333333332, acc:0.9876190476190476, param:31818
f:0.9865142333333332, acc:0.9873333333333333, param:30794
f:0.9865142333333332, acc:0.9873333333333333, param

 0.9878475666666666,
 0.9892380952380952,
 30794]

# Ejecución 2

In [ ]:
## MÓDULOS ##########################################################################################
from Operators import *
from EncodingClass import *
from Decoding import *
from DataReader import *
from DistributedTraining import *

## LIBRERÍAS ########################################################################################
import numpy as np
from torch import optim
import pandas as pd
import timeit
import torch
from torch import nn
import pickle

## HIPERPARÁMETROS DE LA CNN ########################################################################
loss_func = nn.NLLLoss(reduction = "sum") #Defining loss function
lr = 1e-4 #Defining learning rate
#Maximun and minimum numbers of layers to initialize networks
min_conv = 2
max_conv = 5
min_full = 1
max_full = 4
w = 0.3 #penalization weight
max_params = 2e6
num_epochs = 10

## PARÁMETROS GA ####################################################################################
N = 20 #Population size
cr = 0.7 #Crossover rate
mr = 0.7 #Mutation rate
#T = 50 #Number of generations
t_size = 5 #tournament size
train_dl, test_dl = loading_data()
#device1 = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device1 = torch.device("cuda:0")
device1 = torch.device("cuda")

## CREAR POBLACIÓN ##################################################################################
pop = []
bestAcc = []
bestF = []
bestParams = []
print("Iniciando población...\n")
while len(pop) < N:
  while True:
    try:
      acc_list = []
      #Creating genomes (genetic encoding)
      e1 = Encoding(min_conv,max_conv,min_full,max_full)
      #Decoding the network
      network1 = decoding(e1)
      #Creating the CNN
      cnn1 = CNN(e1, network1[0], network1[1], network1[2])
      print("Individuo {} creado:".format(len(pop)+1))
      #Training the CNN
      training('1', device1, cnn1, num_epochs, loss_func,train_dl, test_dl, lr, w, max_params, acc_list)
      #Add individual to population
      pop.append([e1,acc_list[0][1],acc_list[0][2],acc_list[0][3]])
      print("Individuo {} evaluado.\n".format(len(pop)))
      break
    except:
      pass

print("Población creada!!")

Iniciando población...

Individuo 1 creado:
Individuo 1 evaluado.

Individuo 2 creado:
Individuo 2 evaluado.

Individuo 3 creado:
Individuo 3 evaluado.

Individuo 4 creado:
Individuo 4 evaluado.

Individuo 5 creado:
Individuo 5 evaluado.

Individuo 6 creado:
Individuo 6 evaluado.

Individuo 7 creado:
Individuo 7 evaluado.

Individuo 8 creado:
Individuo 8 evaluado.

Individuo 9 creado:
Individuo 9 evaluado.

Individuo 10 creado:
Individuo 10 evaluado.

Individuo 11 creado:
Individuo 11 evaluado.

Individuo 12 creado:
Individuo 12 evaluado.

Individuo 13 creado:
Individuo 13 evaluado.

Individuo 14 creado:
Individuo 14 evaluado.

Individuo 15 creado:
Individuo 15 evaluado.

Individuo 16 creado:
Individuo 16 evaluado.

Individuo 17 creado:
Individuo 17 creado:


/usr/local/lib/python3.7/dist-packages/torch/nn/init.py:403: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Individuo 17 evaluado.

Individuo 18 creado:
Individuo 18 evaluado.

Individuo 19 creado:
Individuo 19 evaluado.

Individuo 20 creado:
Individuo 20 creado:
Individuo 20 evaluado.

Población creada!!


In [ ]:
#Mostrar aptitudes, precisión y num. parámetros de población inicial
for network in pop:
  print("f:{}, acc:{}, param:{}".format(network[1],network[2],network[3]))
max(pop, key = lambda x: x[1])

f:0.8879091333333333, acc:0.855904761904762, param:74828
f:0.8773971999999999, acc:0.826, param:5352
f:0.9625039666666666, acc:0.9842380952380952, param:176418
f:0.7158654666666666, acc:0.5953809523809523, param:6008
f:0.9454268333333332, acc:0.9817619047619047, param:278710
f:0.9771842666666666, acc:0.9766666666666667, param:43216
f:0.7833585333333333, acc:0.6909047619047619, param:1832
f:0.9624712666666666, acc:0.9533809523809523, param:32636
f:0.9634636, acc:0.9502857142857143, param:11576
f:0.9685827999999999, acc:0.9588571428571429, param:17448
f:0.9595140666666666, acc:0.9455238095238095, param:15684
f:0.9771205333333333, acc:0.970047619047619, param:12752
f:0.9598170333333333, acc:0.9467619047619048, param:19442
f:0.9212740666666666, acc:0.9010952380952381, param:63284
f:0.8355645666666667, acc:0.7668095238095238, param:8014
f:0.9586141333333333, acc:0.9421904761904762, param:6128
f:0.9070932333333332, acc:0.8683333333333333, param:4934
f:0.9358768333333332, acc:0.91133333333333

 0.9771842666666666,
 0.9766666666666667,
 43216]

In [ ]:
T = 10 #Number of generations
print("Iniciando genético...")
for t in range(T):
  print('Generation: ', t)
    
  #Parents Selection
  parents = []
  while len(parents) < int(N/2):
    #Tournament Selection
    tournament = random.sample(pop, t_size)
    p1 = selection(tournament, 'max')
    tournament = random.sample(pop, t_size)
    p2 = selection(tournament, 'max')
    while p1 == p2:
      tournament = random.sample(pop, t_size)
      p2 = selection(tournament, 'max')  
        
    parents.append(p1)
    parents.append(p2)
    
    #Reproduction
  offspring = []
  while len(offspring) < int(N/2):
    par = random.sample(parents, 2)
    #Crossover + Mutation
    if cr >= random.uniform(0,1): #Crossover
      p1 = par[0][0]
      p2 = par[1][0]
      c1, c2 = crossover(p1, p2)
            
    #Mutation
    if mr >= random.uniform(0,1):
      mutation(c1)
            
    if mr >= random.uniform(0,1):
      mutation(c2)

    #Try to evaluate the first offspring
    try:
      network1 = decoding(c1)
      cnn1 = CNN(c1, network1[0], network1[1], network1[2])
      acc1_list = []
      training('1', device1, cnn1, num_epochs, loss_func,train_dl, test_dl, lr, w, max_params, acc1_list)
      print("Descendiente {} evaluado.".format(len(offspring)+1))
      offspring.append([c1,acc1_list[0][1],acc1_list[0][2],acc1_list[0][3]])
    except:
      c1_random = random.sample(pop,1)[0]
      print("Descendiente {} aleatorio de pop.".format(len(offspring)+1))
      offspring.append([c1_random[0],c1_random[1],c1_random[2],c1_random[3]])

    #Try to evaluate the second offspring
    try:
      network2 = decoding(c2)
      cnn2 = CNN(c2, network2[0], network2[1], network2[2])
      acc2_list = []
      training('2', device1, cnn2, num_epochs, loss_func,train_dl, test_dl, lr, w, max_params, acc2_list)
      print("Descendiente {} evaluado.".format(len(offspring)+1))
      offspring.append([c2,acc2_list[0][1],acc2_list[0][2],acc2_list[0][3]])
    except:
      c2_random = random.sample(pop,1)[0]
      print("Descendiente {} aleatorio de pop.".format(len(offspring)+1))
      offspring.append([c2_random[0],c2_random[1],c2_random[2],c2_random[3]])

  #Replacement with elitism
  pop = pop + offspring
  pop.sort(reverse = True, key = lambda x: x[1])
  pop = pop[:N]
    
  leader = max(pop, key = lambda x: x[1])
  bestAcc.append(leader[2])
  bestF.append(leader[1])
  bestParams.append(leader[3])
    
        
  print('Best fitness: ', leader[1])
  print('Best accuracy: ', leader[2])
  print('Best No. of Params: ', leader[3])
  print('No. of Conv. Layers: ', leader[0].n_conv)
  print('No. of FC Layers: ', leader[0].n_full)
  print('--------------------------------------------')

print("Fin genético!!")

Iniciando genético...
Generation:  0
Descendiente 1 evaluado.
Descendiente 2 evaluado.
Descendiente 3 evaluado.
Descendiente 4 evaluado.
Descendiente 5 evaluado.
Descendiente 6 evaluado.
Descendiente 7 evaluado.
Descendiente 8 evaluado.
Descendiente 9 evaluado.
Descendiente 10 evaluado.
Best fitness:  0.9771842666666666
Best accuracy:  0.9766666666666667
Best No. of Params:  43216
No. of Conv. Layers:  4
No. of FC Layers:  2
--------------------------------------------
Generation:  1
Descendiente 1 evaluado.
Descendiente 2 evaluado.
Descendiente 3 evaluado.
Descendiente 4 evaluado.
Descendiente 5 evaluado.
Descendiente 6 evaluado.
Descendiente 7 evaluado.
Descendiente 8 evaluado.
Descendiente 9 evaluado.
Descendiente 10 evaluado.
Best fitness:  0.9816875666666667
Best accuracy:  0.9860952380952381
Best No. of Params:  57194
No. of Conv. Layers:  2
No. of FC Layers:  1
--------------------------------------------
Generation:  2
Descendiente 1 evaluado.
Descendiente 2 evaluado.
Descendie

/usr/local/lib/python3.7/dist-packages/torch/nn/init.py:403: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Descendiente 7 evaluado.
Descendiente 8 evaluado.
Descendiente 9 evaluado.
Descendiente 10 evaluado.
Best fitness:  0.9816875666666667
Best accuracy:  0.9860952380952381
Best No. of Params:  57194
No. of Conv. Layers:  2
No. of FC Layers:  1
--------------------------------------------
Generation:  3
Descendiente 1 evaluado.
Descendiente 2 evaluado.
Descendiente 3 evaluado.
Descendiente 4 evaluado.
Descendiente 5 evaluado.
Descendiente 6 evaluado.
Descendiente 7 evaluado.
Descendiente 8 evaluado.
Descendiente 9 evaluado.
Descendiente 10 evaluado.
Best fitness:  0.9839131666666665
Best accuracy:  0.9803809523809524
Best No. of Params:  15690
No. of Conv. Layers:  4
No. of FC Layers:  1
--------------------------------------------
Generation:  4
Descendiente 1 evaluado.
Descendiente 2 evaluado.
Descendiente 3 evaluado.
Descendiente 4 evaluado.
Descendiente 5 evaluado.
Descendiente 6 evaluado.
Descendiente 7 evaluado.
Descendiente 8 aleatorio de pop.
Descendiente 9 evaluado.
Descendiente 

In [ ]:
T = 10 #Number of generations
print("Iniciando genético...")
for t in range(T):
  print('Generation: ', t)
    
  #Parents Selection
  parents = []
  while len(parents) < int(N/2):
    #Tournament Selection
    tournament = random.sample(pop, t_size)
    p1 = selection(tournament, 'max')
    tournament = random.sample(pop, t_size)
    p2 = selection(tournament, 'max')
    while p1 == p2:
      tournament = random.sample(pop, t_size)
      p2 = selection(tournament, 'max')  
        
    parents.append(p1)
    parents.append(p2)
    
    #Reproduction
  offspring = []
  while len(offspring) < int(N/2):
    par = random.sample(parents, 2)
    #Crossover + Mutation
    if cr >= random.uniform(0,1): #Crossover
      p1 = par[0][0]
      p2 = par[1][0]
      c1, c2 = crossover(p1, p2)
            
    #Mutation
    if mr >= random.uniform(0,1):
      mutation(c1)
            
    if mr >= random.uniform(0,1):
      mutation(c2)

    #Try to evaluate the first offspring
    try:
      network1 = decoding(c1)
      cnn1 = CNN(c1, network1[0], network1[1], network1[2])
      acc1_list = []
      training('1', device1, cnn1, num_epochs, loss_func,train_dl, test_dl, lr, w, max_params, acc1_list)
      print("Descendiente {} evaluado.".format(len(offspring)+1))
      offspring.append([c1,acc1_list[0][1],acc1_list[0][2],acc1_list[0][3]])
    except:
      c1_random = random.sample(pop,1)[0]
      print("Descendiente {} aleatorio de pop.".format(len(offspring)+1))
      offspring.append([c1_random[0],c1_random[1],c1_random[2],c1_random[3]])

    #Try to evaluate the second offspring
    try:
      network2 = decoding(c2)
      cnn2 = CNN(c2, network2[0], network2[1], network2[2])
      acc2_list = []
      training('2', device1, cnn2, num_epochs, loss_func,train_dl, test_dl, lr, w, max_params, acc2_list)
      print("Descendiente {} evaluado.".format(len(offspring)+1))
      offspring.append([c2,acc2_list[0][1],acc2_list[0][2],acc2_list[0][3]])
    except:
      c2_random = random.sample(pop,1)[0]
      print("Descendiente {} aleatorio de pop.".format(len(offspring)+1))
      offspring.append([c2_random[0],c2_random[1],c2_random[2],c2_random[3]])

  #Replacement with elitism
  pop = pop + offspring
  pop.sort(reverse = True, key = lambda x: x[1])
  pop = pop[:N]
    
  leader = max(pop, key = lambda x: x[1])
  bestAcc.append(leader[2])
  bestF.append(leader[1])
  bestParams.append(leader[3])
    
        
  print('Best fitness: ', leader[1])
  print('Best accuracy: ', leader[2])
  print('Best No. of Params: ', leader[3])
  print('No. of Conv. Layers: ', leader[0].n_conv)
  print('No. of FC Layers: ', leader[0].n_full)
  print('--------------------------------------------')

print("Fin genético!!")

Iniciando genético...
Generation:  0
Descendiente 1 evaluado.
Descendiente 2 evaluado.
Descendiente 3 evaluado.
Descendiente 4 aleatorio de pop.


/usr/local/lib/python3.7/dist-packages/torch/nn/init.py:403: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Descendiente 5 evaluado.
Descendiente 6 aleatorio de pop.
Descendiente 7 evaluado.
Descendiente 8 evaluado.
Descendiente 9 evaluado.
Descendiente 10 aleatorio de pop.
Best fitness:  0.9867858333333334
Best accuracy:  0.984904761904762
Best No. of Params:  17650
No. of Conv. Layers:  4
No. of FC Layers:  1
--------------------------------------------
Generation:  1
Descendiente 1 evaluado.
Descendiente 2 aleatorio de pop.
Descendiente 3 evaluado.
Descendiente 4 aleatorio de pop.
Descendiente 5 evaluado.
Descendiente 6 evaluado.
Descendiente 7 evaluado.
Descendiente 8 evaluado.
Descendiente 9 evaluado.
Descendiente 10 evaluado.
Best fitness:  0.9867858333333334
Best accuracy:  0.984904761904762
Best No. of Params:  17650
No. of Conv. Layers:  4
No. of FC Layers:  1
--------------------------------------------
Generation:  2
Descendiente 1 evaluado.
Descendiente 2 evaluado.
Descendiente 3 evaluado.
Descendiente 4 evaluado.
Descendiente 5 evaluado.
Descendiente 6 evaluado.
Descendiente 7 e

In [ ]:
#Mostrar aptitudes, precisión y num. parámetros de población final
for network in pop:
  print("f:{}, acc:{}, param:{}".format(network[1],network[2],network[3]))
max(pop, key = lambda x: x[1])

f:0.9867858333333334, acc:0.984904761904762, param:17650
f:0.9866525, acc:0.9847142857142858, param:17650
f:0.9866525, acc:0.9847142857142858, param:17650
f:0.9864258333333333, acc:0.984904761904762, param:20050
f:0.9863858333333333, acc:0.9843333333333333, param:17650
f:0.9863858333333333, acc:0.9843333333333333, param:17650
f:0.9862191666666666, acc:0.9840952380952381, param:17650
f:0.9862191666666666, acc:0.9840952380952381, param:17650
f:0.9862191666666666, acc:0.9840952380952381, param:17650
f:0.9859191666666667, acc:0.9836666666666667, param:17650
f:0.9856525, acc:0.9832857142857143, param:17650
f:0.9856525, acc:0.9832857142857143, param:17650
f:0.9855719666666667, acc:0.9826666666666667, param:15298
f:0.9855719666666667, acc:0.9826666666666667, param:15298
f:0.9854525, acc:0.983, param:17650
f:0.9854525, acc:0.983, param:17650
f:0.9854525, acc:0.983, param:17650
f:0.9854525, acc:0.983, param:17650
f:0.9854525, acc:0.983, param:17650
f:0.9854525, acc:0.983, param:17650


 0.9867858333333334,
 0.984904761904762,
 17650]

# Ejecución 3

In [ ]:
## MÓDULOS ##########################################################################################
from Operators import *
from EncodingClass import *
from Decoding import *
from DataReader import *
from DistributedTraining import *

## LIBRERÍAS ########################################################################################
import numpy as np
from torch import optim
import pandas as pd
import timeit
import torch
from torch import nn
import pickle

## HIPERPARÁMETROS DE LA CNN ########################################################################
loss_func = nn.NLLLoss(reduction = "sum") #Defining loss function
lr = 1e-4 #Defining learning rate
#Maximun and minimum numbers of layers to initialize networks
min_conv = 2
max_conv = 5
min_full = 1
max_full = 4
w = 0.3 #penalization weight
max_params = 2e6
num_epochs = 10

## PARÁMETROS GA ####################################################################################
N = 20 #Population size
cr = 0.7 #Crossover rate
mr = 0.7 #Mutation rate
#T = 50 #Number of generations
t_size = 5 #tournament size
train_dl, test_dl = loading_data()
#device1 = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device1 = torch.device("cuda:0")
device1 = torch.device("cuda")

## CREAR POBLACIÓN ##################################################################################
pop = []
bestAcc = []
bestF = []
bestParams = []
print("Iniciando población...\n")
while len(pop) < N:
  while True:
    try:
      acc_list = []
      #Creating genomes (genetic encoding)
      e1 = Encoding(min_conv,max_conv,min_full,max_full)
      #Decoding the network
      network1 = decoding(e1)
      #Creating the CNN
      cnn1 = CNN(e1, network1[0], network1[1], network1[2])
      print("Individuo {} creado:".format(len(pop)+1))
      #Training the CNN
      training('1', device1, cnn1, num_epochs, loss_func,train_dl, test_dl, lr, w, max_params, acc_list)
      #Add individual to population
      pop.append([e1,acc_list[0][1],acc_list[0][2],acc_list[0][3]])
      print("Individuo {} evaluado.\n".format(len(pop)))
      break
    except:
      pass

print("Población creada!!")

Iniciando población...

Individuo 1 creado:
Individuo 1 evaluado.

Individuo 2 creado:
Individuo 2 evaluado.

Individuo 3 creado:
Individuo 3 evaluado.

Individuo 4 creado:
Individuo 4 evaluado.

Individuo 5 creado:
Individuo 5 evaluado.

Individuo 6 creado:
Individuo 6 evaluado.

Individuo 7 creado:
Individuo 7 evaluado.

Individuo 8 creado:
Individuo 8 evaluado.

Individuo 9 creado:
Individuo 9 evaluado.

Individuo 10 creado:
Individuo 10 evaluado.

Individuo 11 creado:
Individuo 11 evaluado.

Individuo 12 creado:
Individuo 12 evaluado.

Individuo 13 creado:
Individuo 13 evaluado.

Individuo 14 creado:
Individuo 14 evaluado.

Individuo 15 creado:
Individuo 15 evaluado.

Individuo 16 creado:
Individuo 16 evaluado.

Individuo 17 creado:
Individuo 17 evaluado.

Individuo 18 creado:
Individuo 18 evaluado.

Individuo 19 creado:
Individuo 19 evaluado.

Individuo 20 creado:
Individuo 20 evaluado.

Población creada!!


In [ ]:
#Mostrar aptitudes, precisión y num. parámetros de población inicial
for network in pop:
  print("f:{}, acc:{}, param:{}".format(network[1],network[2],network[3]))
max(pop, key = lambda x: x[1])

f:0.8876037999999999, acc:0.8508571428571429, param:53308
f:0.9705168, acc:0.9604285714285714, param:11888
f:0.9288730333333333, acc:0.9061904761904762, param:36402
f:0.9734791999999999, acc:0.9658571428571429, param:17472
f:0.9486749999999999, acc:0.9274285714285714, param:3500
f:0.8444673666666667, acc:0.7938095238095239, param:74662
f:0.9804151999999999, acc:0.98, param:37232
f:0.8276753666666665, acc:0.7615238095238095, param:35942
f:0.9486977, acc:0.9877142857142858, param:284682
f:0.9627499666666666, acc:0.9563809523809523, param:44778
f:0.9779667666666667, acc:0.9708095238095238, param:10666
f:0.9779472333333333, acc:0.972047619047619, param:16574
f:0.36091843333333334, acc:0.09704761904761905, param:46766
f:0.7901495333333333, acc:0.7057619047619048, param:25892
f:0.9678144999999999, acc:0.9558571428571428, param:8570
f:0.7665999666666666, acc:0.6673809523809524, param:3778
f:0.975741, acc:0.9668571428571429, param:7060
f:0.9474366666666666, acc:0.927952380952381, param:14200
f

[<EncodingClass.Encoding at 0x7f6243cac410>, 0.9804151999999999, 0.98, 37232]

In [ ]:
T = 10 #Number of generations
print("Iniciando genético...")
for t in range(T):
  print('Generation: ', t)
    
  #Parents Selection
  parents = []
  while len(parents) < int(N/2):
    #Tournament Selection
    tournament = random.sample(pop, t_size)
    p1 = selection(tournament, 'max')
    tournament = random.sample(pop, t_size)
    p2 = selection(tournament, 'max')
    while p1 == p2:
      tournament = random.sample(pop, t_size)
      p2 = selection(tournament, 'max')  
        
    parents.append(p1)
    parents.append(p2)
    
    #Reproduction
  offspring = []
  while len(offspring) < int(N/2):
    par = random.sample(parents, 2)
    #Crossover + Mutation
    if cr >= random.uniform(0,1): #Crossover
      p1 = par[0][0]
      p2 = par[1][0]
      c1, c2 = crossover(p1, p2)
            
    #Mutation
    if mr >= random.uniform(0,1):
      mutation(c1)
            
    if mr >= random.uniform(0,1):
      mutation(c2)

    #Try to evaluate the first offspring
    try:
      network1 = decoding(c1)
      cnn1 = CNN(c1, network1[0], network1[1], network1[2])
      acc1_list = []
      training('1', device1, cnn1, num_epochs, loss_func,train_dl, test_dl, lr, w, max_params, acc1_list)
      print("Descendiente {} evaluado.".format(len(offspring)+1))
      offspring.append([c1,acc1_list[0][1],acc1_list[0][2],acc1_list[0][3]])
    except:
      c1_random = random.sample(pop,1)[0]
      print("Descendiente {} aleatorio de pop.".format(len(offspring)+1))
      offspring.append([c1_random[0],c1_random[1],c1_random[2],c1_random[3]])

    #Try to evaluate the second offspring
    try:
      network2 = decoding(c2)
      cnn2 = CNN(c2, network2[0], network2[1], network2[2])
      acc2_list = []
      training('2', device1, cnn2, num_epochs, loss_func,train_dl, test_dl, lr, w, max_params, acc2_list)
      print("Descendiente {} evaluado.".format(len(offspring)+1))
      offspring.append([c2,acc2_list[0][1],acc2_list[0][2],acc2_list[0][3]])
    except:
      c2_random = random.sample(pop,1)[0]
      print("Descendiente {} aleatorio de pop.".format(len(offspring)+1))
      offspring.append([c2_random[0],c2_random[1],c2_random[2],c2_random[3]])

  #Replacement with elitism
  pop = pop + offspring
  pop.sort(reverse = True, key = lambda x: x[1])
  pop = pop[:N]
    
  leader = max(pop, key = lambda x: x[1])
  bestAcc.append(leader[2])
  bestF.append(leader[1])
  bestParams.append(leader[3])
    
        
  print('Best fitness: ', leader[1])
  print('Best accuracy: ', leader[2])
  print('Best No. of Params: ', leader[3])
  print('No. of Conv. Layers: ', leader[0].n_conv)
  print('No. of FC Layers: ', leader[0].n_full)
  print('--------------------------------------------')

print("Fin genético!!")

Iniciando genético...
Generation:  0
Descendiente 1 evaluado.
Descendiente 2 evaluado.
Descendiente 3 evaluado.
Descendiente 4 evaluado.
Descendiente 5 evaluado.
Descendiente 6 evaluado.
Descendiente 7 evaluado.
Descendiente 8 evaluado.
Descendiente 9 evaluado.
Descendiente 10 evaluado.
Best fitness:  0.9812437333333333
Best accuracy:  0.9807619047619047
Best No. of Params:  35264
No. of Conv. Layers:  4
No. of FC Layers:  2
--------------------------------------------
Generation:  1
Descendiente 1 evaluado.
Descendiente 2 evaluado.
Descendiente 3 evaluado.
Descendiente 4 evaluado.
Descendiente 5 evaluado.
Descendiente 6 evaluado.
Descendiente 7 evaluado.
Descendiente 8 evaluado.
Descendiente 9 evaluado.
Descendiente 10 evaluado.
Best fitness:  0.9821219333333333
Best accuracy:  0.9779047619047619
Best No. of Params:  16076
No. of Conv. Layers:  5
No. of FC Layers:  3
--------------------------------------------
Generation:  2
Descendiente 1 evaluado.
Descendiente 2 evaluado.
Descendie

/usr/local/lib/python3.7/dist-packages/torch/nn/init.py:403: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Descendiente 6 evaluado.
Descendiente 7 evaluado.
Descendiente 8 evaluado.
Descendiente 9 evaluado.
Descendiente 10 evaluado.
Best fitness:  0.9840766666666667
Best accuracy:  0.9846666666666667
Best No. of Params:  34600
No. of Conv. Layers:  4
No. of FC Layers:  2
--------------------------------------------
Generation:  6
Descendiente 1 evaluado.
Descendiente 2 evaluado.
Descendiente 3 evaluado.
Descendiente 4 evaluado.
Descendiente 5 aleatorio de pop.
Descendiente 6 evaluado.
Descendiente 7 aleatorio de pop.
Descendiente 8 evaluado.
Descendiente 9 evaluado.
Descendiente 10 evaluado.
Best fitness:  0.9840766666666667
Best accuracy:  0.9846666666666667
Best No. of Params:  34600
No. of Conv. Layers:  4
No. of FC Layers:  2
--------------------------------------------
Generation:  7
Descendiente 1 evaluado.
Descendiente 2 evaluado.
Descendiente 3 evaluado.
Descendiente 4 evaluado.
Descendiente 5 evaluado.
Descendiente 6 evaluado.
Descendiente 7 evaluado.
Descendiente 8 evaluado.
Desce

In [ ]:
T = 10 #Number of generations
print("Iniciando genético...")
for t in range(T):
  print('Generation: ', t)
    
  #Parents Selection
  parents = []
  while len(parents) < int(N/2):
    #Tournament Selection
    tournament = random.sample(pop, t_size)
    p1 = selection(tournament, 'max')
    tournament = random.sample(pop, t_size)
    p2 = selection(tournament, 'max')
    while p1 == p2:
      tournament = random.sample(pop, t_size)
      p2 = selection(tournament, 'max')  
        
    parents.append(p1)
    parents.append(p2)
    
    #Reproduction
  offspring = []
  while len(offspring) < int(N/2):
    par = random.sample(parents, 2)
    #Crossover + Mutation
    if cr >= random.uniform(0,1): #Crossover
      p1 = par[0][0]
      p2 = par[1][0]
      c1, c2 = crossover(p1, p2)
            
    #Mutation
    if mr >= random.uniform(0,1):
      mutation(c1)
            
    if mr >= random.uniform(0,1):
      mutation(c2)

    #Try to evaluate the first offspring
    try:
      network1 = decoding(c1)
      cnn1 = CNN(c1, network1[0], network1[1], network1[2])
      acc1_list = []
      training('1', device1, cnn1, num_epochs, loss_func,train_dl, test_dl, lr, w, max_params, acc1_list)
      print("Descendiente {} evaluado.".format(len(offspring)+1))
      offspring.append([c1,acc1_list[0][1],acc1_list[0][2],acc1_list[0][3]])
    except:
      c1_random = random.sample(pop,1)[0]
      print("Descendiente {} aleatorio de pop.".format(len(offspring)+1))
      offspring.append([c1_random[0],c1_random[1],c1_random[2],c1_random[3]])

    #Try to evaluate the second offspring
    try:
      network2 = decoding(c2)
      cnn2 = CNN(c2, network2[0], network2[1], network2[2])
      acc2_list = []
      training('2', device1, cnn2, num_epochs, loss_func,train_dl, test_dl, lr, w, max_params, acc2_list)
      print("Descendiente {} evaluado.".format(len(offspring)+1))
      offspring.append([c2,acc2_list[0][1],acc2_list[0][2],acc2_list[0][3]])
    except:
      c2_random = random.sample(pop,1)[0]
      print("Descendiente {} aleatorio de pop.".format(len(offspring)+1))
      offspring.append([c2_random[0],c2_random[1],c2_random[2],c2_random[3]])

  #Replacement with elitism
  pop = pop + offspring
  pop.sort(reverse = True, key = lambda x: x[1])
  pop = pop[:N]
    
  leader = max(pop, key = lambda x: x[1])
  bestAcc.append(leader[2])
  bestF.append(leader[1])
  bestParams.append(leader[3])
    
        
  print('Best fitness: ', leader[1])
  print('Best accuracy: ', leader[2])
  print('Best No. of Params: ', leader[3])
  print('No. of Conv. Layers: ', leader[0].n_conv)
  print('No. of FC Layers: ', leader[0].n_full)
  print('--------------------------------------------')

print("Fin genético!!")

Iniciando genético...
Generation:  0
Descendiente 1 evaluado.
Descendiente 2 evaluado.
Descendiente 3 evaluado.
Descendiente 4 aleatorio de pop.


/usr/local/lib/python3.7/dist-packages/torch/nn/init.py:403: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Descendiente 5 evaluado.
Descendiente 6 aleatorio de pop.
Descendiente 7 evaluado.
Descendiente 8 aleatorio de pop.
Descendiente 9 aleatorio de pop.
Descendiente 10 evaluado.
Best fitness:  0.9853334666666667
Best accuracy:  0.9852380952380952
Best No. of Params:  28888
No. of Conv. Layers:  5
No. of FC Layers:  2
--------------------------------------------
Generation:  1
Descendiente 1 evaluado.
Descendiente 2 evaluado.
Descendiente 3 evaluado.
Descendiente 4 evaluado.
Descendiente 5 evaluado.
Descendiente 6 evaluado.
Descendiente 7 evaluado.
Descendiente 8 evaluado.
Descendiente 9 evaluado.
Descendiente 10 evaluado.
Best fitness:  0.9853334666666667
Best accuracy:  0.9852380952380952
Best No. of Params:  28888
No. of Conv. Layers:  5
No. of FC Layers:  2
--------------------------------------------
Generation:  2
Descendiente 1 evaluado.
Descendiente 2 evaluado.
Descendiente 3 evaluado.
Descendiente 4 evaluado.
Descendiente 5 evaluado.
Descendiente 6 evaluado.
Descendiente 7 evaluad

In [ ]:
#Mostrar aptitudes, precisión y num. parámetros de población final
for network in pop:
  print("f:{}, acc:{}, param:{}".format(network[1],network[2],network[3]))
max(pop, key = lambda x: x[1])

f:0.9853919999999999, acc:0.9844285714285714, param:24720
f:0.9853334666666667, acc:0.9852380952380952, param:28888
f:0.9849034333333333, acc:0.9867619047619047, param:38866
f:0.9848233333333332, acc:0.9846190476190476, param:29400
f:0.9847586666666667, acc:0.9835238095238096, param:24720
f:0.9847282666666666, acc:0.9855238095238095, param:34256
f:0.9847282666666666, acc:0.9855238095238095, param:34256
f:0.9847282666666666, acc:0.9855238095238095, param:34256
f:0.984692, acc:0.9834285714285714, param:24720
f:0.984692, acc:0.9834285714285714, param:24720
f:0.984692, acc:0.9834285714285714, param:24720
f:0.984692, acc:0.9834285714285714, param:24720
f:0.984692, acc:0.9834285714285714, param:24720
f:0.9846161333333332, acc:0.9847619047619047, param:31448
f:0.9845818666666666, acc:0.9850952380952381, param:33232
f:0.9845818666666666, acc:0.9850952380952381, param:33232
f:0.9845633333333332, acc:0.9826190476190476, param:21800
f:0.9845566666666665, acc:0.9842380952380952, param:29400
f:0.98

 0.9853919999999999,
 0.9844285714285714,
 24720]

# Ejecución 4

In [ ]:
## MÓDULOS ##########################################################################################
from Operators import *
from EncodingClass import *
from Decoding import *
from DataReader import *
from DistributedTraining import *

## LIBRERÍAS ########################################################################################
import numpy as np
from torch import optim
import pandas as pd
import timeit
import torch
from torch import nn
import pickle

## HIPERPARÁMETROS DE LA CNN ########################################################################
loss_func = nn.NLLLoss(reduction = "sum") #Defining loss function
lr = 1e-4 #Defining learning rate
#Maximun and minimum numbers of layers to initialize networks
min_conv = 2
max_conv = 5
min_full = 1
max_full = 4
w = 0.3 #penalization weight
max_params = 2e6
num_epochs = 10

## PARÁMETROS GA ####################################################################################
N = 20 #Population size
cr = 0.7 #Crossover rate
mr = 0.7 #Mutation rate
#T = 50 #Number of generations
t_size = 5 #tournament size
train_dl, test_dl = loading_data()
#device1 = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device1 = torch.device("cuda:0")
device1 = torch.device("cuda")

## CREAR POBLACIÓN ##################################################################################
pop = []
bestAcc = []
bestF = []
bestParams = []
print("Iniciando población...\n")
while len(pop) < N:
  while True:
    try:
      acc_list = []
      #Creating genomes (genetic encoding)
      e1 = Encoding(min_conv,max_conv,min_full,max_full)
      #Decoding the network
      network1 = decoding(e1)
      #Creating the CNN
      cnn1 = CNN(e1, network1[0], network1[1], network1[2])
      print("Individuo {} creado:".format(len(pop)+1))
      #Training the CNN
      training('1', device1, cnn1, num_epochs, loss_func,train_dl, test_dl, lr, w, max_params, acc_list)
      #Add individual to population
      pop.append([e1,acc_list[0][1],acc_list[0][2],acc_list[0][3]])
      print("Individuo {} evaluado.\n".format(len(pop)))
      break
    except:
      pass

print("Población creada!!")

Iniciando población...

Individuo 1 creado:
Individuo 1 evaluado.

Individuo 2 creado:
Individuo 2 evaluado.

Individuo 3 creado:
Individuo 3 evaluado.

Individuo 4 creado:
Individuo 4 evaluado.

Individuo 5 creado:
Individuo 5 evaluado.

Individuo 6 creado:
Individuo 6 creado:


/usr/local/lib/python3.7/dist-packages/torch/nn/init.py:403: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Individuo 6 evaluado.

Individuo 7 creado:
Individuo 7 evaluado.

Individuo 8 creado:
Individuo 8 evaluado.

Individuo 9 creado:
Individuo 9 evaluado.

Individuo 10 creado:
Individuo 10 evaluado.

Individuo 11 creado:
Individuo 11 creado:
Individuo 11 evaluado.

Individuo 12 creado:
Individuo 12 evaluado.

Individuo 13 creado:
Individuo 13 evaluado.

Individuo 14 creado:
Individuo 14 evaluado.

Individuo 15 creado:
Individuo 15 evaluado.

Individuo 16 creado:
Individuo 16 evaluado.

Individuo 17 creado:
Individuo 17 creado:
Individuo 17 evaluado.

Individuo 18 creado:
Individuo 18 evaluado.

Individuo 19 creado:
Individuo 19 evaluado.

Individuo 20 creado:
Individuo 20 evaluado.

Población creada!!


In [ ]:
#Mostrar aptitudes, precisión y num. parámetros de población inicial
for network in pop:
  print("f:{}, acc:{}, param:{}".format(network[1],network[2],network[3]))
max(pop, key = lambda x: x[1])

f:0.8831782666666665, acc:0.8333809523809523, param:1256
f:0.9621265666666667, acc:0.958952380952381, param:60934
f:0.9716526666666667, acc:0.9652380952380952, param:26760
f:0.9032811, acc:0.8674285714285714, param:26126
f:0.9809596999999999, acc:0.989, param:75602
f:0.7903317999999999, acc:0.703, param:11788
f:0.9645078333333332, acc:0.9621904761904762, param:60170
f:0.9627543666666666, acc:0.9742380952380952, param:128082
f:0.9736967999999999, acc:0.9668571428571429, param:20688
f:0.9368837333333333, acc:0.911047619047619, param:5664
f:0.9677790999999999, acc:0.9588571428571429, param:22806
f:0.9738993666666665, acc:0.9648095238095238, param:9782
f:0.5527949, acc:0.3615714285714286, param:2034
f:0.9453037, acc:0.9232857142857143, param:6642
f:0.9631774666666666, acc:0.9503809523809523, param:13928
f:0.9828118333333333, acc:0.979904761904762, param:20810
f:0.9790514666666665, acc:0.9779523809523809, param:36768
f:0.9772714333333334, acc:0.9743333333333334, param:31746
f:0.729407633333

 0.9828118333333333,
 0.979904761904762,
 20810]

In [ ]:
T = 10 #Number of generations
print("Iniciando genético...")
for t in range(T):
  print('Generation: ', t)
    
  #Parents Selection
  parents = []
  while len(parents) < int(N/2):
    #Tournament Selection
    tournament = random.sample(pop, t_size)
    p1 = selection(tournament, 'max')
    tournament = random.sample(pop, t_size)
    p2 = selection(tournament, 'max')
    while p1 == p2:
      tournament = random.sample(pop, t_size)
      p2 = selection(tournament, 'max')  
        
    parents.append(p1)
    parents.append(p2)
    
    #Reproduction
  offspring = []
  while len(offspring) < int(N/2):
    par = random.sample(parents, 2)
    #Crossover + Mutation
    if cr >= random.uniform(0,1): #Crossover
      p1 = par[0][0]
      p2 = par[1][0]
      c1, c2 = crossover(p1, p2)
            
    #Mutation
    if mr >= random.uniform(0,1):
      mutation(c1)
            
    if mr >= random.uniform(0,1):
      mutation(c2)

    #Try to evaluate the first offspring
    try:
      network1 = decoding(c1)
      cnn1 = CNN(c1, network1[0], network1[1], network1[2])
      acc1_list = []
      training('1', device1, cnn1, num_epochs, loss_func,train_dl, test_dl, lr, w, max_params, acc1_list)
      print("Descendiente {} evaluado.".format(len(offspring)+1))
      offspring.append([c1,acc1_list[0][1],acc1_list[0][2],acc1_list[0][3]])
    except:
      c1_random = random.sample(pop,1)[0]
      print("Descendiente {} aleatorio de pop.".format(len(offspring)+1))
      offspring.append([c1_random[0],c1_random[1],c1_random[2],c1_random[3]])

    #Try to evaluate the second offspring
    try:
      network2 = decoding(c2)
      cnn2 = CNN(c2, network2[0], network2[1], network2[2])
      acc2_list = []
      training('2', device1, cnn2, num_epochs, loss_func,train_dl, test_dl, lr, w, max_params, acc2_list)
      print("Descendiente {} evaluado.".format(len(offspring)+1))
      offspring.append([c2,acc2_list[0][1],acc2_list[0][2],acc2_list[0][3]])
    except:
      c2_random = random.sample(pop,1)[0]
      print("Descendiente {} aleatorio de pop.".format(len(offspring)+1))
      offspring.append([c2_random[0],c2_random[1],c2_random[2],c2_random[3]])

  #Replacement with elitism
  pop = pop + offspring
  pop.sort(reverse = True, key = lambda x: x[1])
  pop = pop[:N]
    
  leader = max(pop, key = lambda x: x[1])
  bestAcc.append(leader[2])
  bestF.append(leader[1])
  bestParams.append(leader[3])
    
        
  print('Best fitness: ', leader[1])
  print('Best accuracy: ', leader[2])
  print('Best No. of Params: ', leader[3])
  print('No. of Conv. Layers: ', leader[0].n_conv)
  print('No. of FC Layers: ', leader[0].n_full)
  print('--------------------------------------------')

print("Fin genético!!")

Iniciando genético...
Generation:  0
Descendiente 1 aleatorio de pop.
Descendiente 2 aleatorio de pop.
Descendiente 3 aleatorio de pop.


/usr/local/lib/python3.7/dist-packages/torch/nn/init.py:403: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Descendiente 4 evaluado.
Descendiente 5 evaluado.
Descendiente 6 evaluado.
Descendiente 7 evaluado.
Descendiente 8 evaluado.
Descendiente 9 evaluado.
Descendiente 10 evaluado.
Best fitness:  0.9828118333333333
Best accuracy:  0.979904761904762
Best No. of Params:  20810
No. of Conv. Layers:  4
No. of FC Layers:  1
--------------------------------------------
Generation:  1
Descendiente 1 evaluado.
Descendiente 2 evaluado.
Descendiente 3 evaluado.
Descendiente 4 evaluado.
Descendiente 5 evaluado.
Descendiente 6 evaluado.
Descendiente 7 evaluado.
Descendiente 8 evaluado.
Descendiente 9 evaluado.
Descendiente 10 evaluado.
Best fitness:  0.9850112999999999
Best accuracy:  0.9827142857142858
Best No. of Params:  19258
No. of Conv. Layers:  3
No. of FC Layers:  2
--------------------------------------------
Generation:  2
Descendiente 1 aleatorio de pop.
Descendiente 2 evaluado.
Descendiente 3 aleatorio de pop.
Descendiente 4 evaluado.
Descendiente 5 evaluado.
Descendiente 6 evaluado.
Descen

In [ ]:
T = 10 #Number of generations
print("Iniciando genético...")
for t in range(T):
  print('Generation: ', t)
    
  #Parents Selection
  parents = []
  while len(parents) < int(N/2):
    #Tournament Selection
    tournament = random.sample(pop, t_size)
    p1 = selection(tournament, 'max')
    tournament = random.sample(pop, t_size)
    p2 = selection(tournament, 'max')
    while p1 == p2:
      tournament = random.sample(pop, t_size)
      p2 = selection(tournament, 'max')  
        
    parents.append(p1)
    parents.append(p2)
    
    #Reproduction
  offspring = []
  while len(offspring) < int(N/2):
    par = random.sample(parents, 2)
    #Crossover + Mutation
    if cr >= random.uniform(0,1): #Crossover
      p1 = par[0][0]
      p2 = par[1][0]
      c1, c2 = crossover(p1, p2)
            
    #Mutation
    if mr >= random.uniform(0,1):
      mutation(c1)
            
    if mr >= random.uniform(0,1):
      mutation(c2)

    #Try to evaluate the first offspring
    try:
      network1 = decoding(c1)
      cnn1 = CNN(c1, network1[0], network1[1], network1[2])
      acc1_list = []
      training('1', device1, cnn1, num_epochs, loss_func,train_dl, test_dl, lr, w, max_params, acc1_list)
      print("Descendiente {} evaluado.".format(len(offspring)+1))
      offspring.append([c1,acc1_list[0][1],acc1_list[0][2],acc1_list[0][3]])
    except:
      c1_random = random.sample(pop,1)[0]
      print("Descendiente {} aleatorio de pop.".format(len(offspring)+1))
      offspring.append([c1_random[0],c1_random[1],c1_random[2],c1_random[3]])

    #Try to evaluate the second offspring
    try:
      network2 = decoding(c2)
      cnn2 = CNN(c2, network2[0], network2[1], network2[2])
      acc2_list = []
      training('2', device1, cnn2, num_epochs, loss_func,train_dl, test_dl, lr, w, max_params, acc2_list)
      print("Descendiente {} evaluado.".format(len(offspring)+1))
      offspring.append([c2,acc2_list[0][1],acc2_list[0][2],acc2_list[0][3]])
    except:
      c2_random = random.sample(pop,1)[0]
      print("Descendiente {} aleatorio de pop.".format(len(offspring)+1))
      offspring.append([c2_random[0],c2_random[1],c2_random[2],c2_random[3]])

  #Replacement with elitism
  pop = pop + offspring
  pop.sort(reverse = True, key = lambda x: x[1])
  pop = pop[:N]
    
  leader = max(pop, key = lambda x: x[1])
  bestAcc.append(leader[2])
  bestF.append(leader[1])
  bestParams.append(leader[3])
    
        
  print('Best fitness: ', leader[1])
  print('Best accuracy: ', leader[2])
  print('Best No. of Params: ', leader[3])
  print('No. of Conv. Layers: ', leader[0].n_conv)
  print('No. of FC Layers: ', leader[0].n_full)
  print('--------------------------------------------')

print("Fin genético!!")

Iniciando genético...
Generation:  0
Descendiente 1 evaluado.
Descendiente 2 evaluado.
Descendiente 3 evaluado.
Descendiente 4 evaluado.
Descendiente 5 evaluado.
Descendiente 6 evaluado.
Descendiente 7 evaluado.
Descendiente 8 evaluado.
Descendiente 9 evaluado.
Descendiente 10 evaluado.
Best fitness:  0.9863446333333332
Best accuracy:  0.9846190476190476
Best No. of Params:  19258
No. of Conv. Layers:  3
No. of FC Layers:  2
--------------------------------------------
Generation:  1
Descendiente 1 evaluado.
Descendiente 2 evaluado.
Descendiente 3 evaluado.
Descendiente 4 evaluado.
Descendiente 5 evaluado.
Descendiente 6 evaluado.
Descendiente 7 evaluado.
Descendiente 8 evaluado.
Descendiente 9 evaluado.
Descendiente 10 evaluado.
Best fitness:  0.9863446333333332
Best accuracy:  0.9846190476190476
Best No. of Params:  19258
No. of Conv. Layers:  3
No. of FC Layers:  2
--------------------------------------------
Generation:  2
Descendiente 1 evaluado.
Descendiente 2 evaluado.
Descendie

/usr/local/lib/python3.7/dist-packages/torch/nn/init.py:403: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Descendiente 6 evaluado.
Descendiente 7 evaluado.
Descendiente 8 evaluado.
Descendiente 9 evaluado.
Descendiente 10 aleatorio de pop.
Best fitness:  0.9863446333333332
Best accuracy:  0.9846190476190476
Best No. of Params:  19258
No. of Conv. Layers:  3
No. of FC Layers:  2
--------------------------------------------
Generation:  4
Descendiente 1 evaluado.
Descendiente 2 evaluado.
Descendiente 3 evaluado.
Descendiente 4 evaluado.
Descendiente 5 evaluado.
Descendiente 6 evaluado.
Descendiente 7 evaluado.
Descendiente 8 evaluado.
Descendiente 9 evaluado.
Descendiente 10 evaluado.
Best fitness:  0.9863446333333332
Best accuracy:  0.9846190476190476
Best No. of Params:  19258
No. of Conv. Layers:  3
No. of FC Layers:  2
--------------------------------------------
Generation:  5
Descendiente 1 evaluado.
Descendiente 2 evaluado.
Descendiente 3 evaluado.
Descendiente 4 evaluado.
Descendiente 5 evaluado.
Descendiente 6 evaluado.
Descendiente 7 aleatorio de pop.
Descendiente 8 evaluado.
Desce

In [ ]:
#Mostrar aptitudes, precisión y num. parámetros de población final
for network in pop:
  print("f:{}, acc:{}, param:{}".format(network[1],network[2],network[3]))
max(pop, key = lambda x: x[1])

f:0.9865955666666666, acc:0.9852380952380952, param:20474
f:0.9863446333333332, acc:0.9846190476190476, param:19258
f:0.9863446333333332, acc:0.9846190476190476, param:19258
f:0.9862289, acc:0.9847142857142858, param:20474
f:0.9859830333333333, acc:0.9857619047619047, param:27002
f:0.9858745, acc:0.9837142857142858, param:18170
f:0.9858078333333333, acc:0.9836190476190476, param:18170
f:0.9857459666666666, acc:0.9886666666666667, param:42138
f:0.9857078333333333, acc:0.9834761904761905, param:18170
f:0.9856801, acc:0.9832857142857143, param:17466
f:0.9855955666666667, acc:0.9838095238095238, param:20474
f:0.9854902333333333, acc:0.982904761904762, param:16954
f:0.9854783666666667, acc:0.9835238095238096, param:19922
f:0.9854446333333332, acc:0.9833333333333333, param:19258
f:0.9854235666666666, acc:0.9828095238095238, param:16954
f:0.9853569, acc:0.9827142857142858, param:16954
f:0.9853569, acc:0.9827142857142858, param:16954
f:0.9853569, acc:0.9827142857142858, param:16954
f:0.9853569

 0.9865955666666666,
 0.9852380952380952,
 20474]

# Ejecución 5

In [6]:
## MÓDULOS ##########################################################################################
from Operators import *
from EncodingClass import *
from Decoding import *
from DataReader import *
from DistributedTraining import *

## LIBRERÍAS ########################################################################################
import numpy as np
from torch import optim
import pandas as pd
import timeit
import torch
from torch import nn
import pickle

## HIPERPARÁMETROS DE LA CNN ########################################################################
loss_func = nn.NLLLoss(reduction = "sum") #Defining loss function
lr = 1e-4 #Defining learning rate
#Maximun and minimum numbers of layers to initialize networks
min_conv = 2
max_conv = 5
min_full = 1
max_full = 4
w = 0.3 #penalization weight
max_params = 2e6
num_epochs = 10

## PARÁMETROS GA ####################################################################################
N = 20 #Population size
cr = 0.7 #Crossover rate
mr = 0.7 #Mutation rate
#T = 50 #Number of generations
t_size = 5 #tournament size
train_dl, test_dl = loading_data()
#device1 = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device1 = torch.device("cuda:0")
device1 = torch.device("cuda")

## CREAR POBLACIÓN ##################################################################################
pop = []
bestAcc = []
bestF = []
bestParams = []
print("Iniciando población...\n")
while len(pop) < N:
  while True:
    try:
      acc_list = []
      #Creating genomes (genetic encoding)
      e1 = Encoding(min_conv,max_conv,min_full,max_full)
      #Decoding the network
      network1 = decoding(e1)
      #Creating the CNN
      cnn1 = CNN(e1, network1[0], network1[1], network1[2])
      print("Individuo {} creado:".format(len(pop)+1))
      #Training the CNN
      training('1', device1, cnn1, num_epochs, loss_func,train_dl, test_dl, lr, w, max_params, acc_list)
      #Add individual to population
      pop.append([e1,acc_list[0][1],acc_list[0][2],acc_list[0][3]])
      print("Individuo {} evaluado.\n".format(len(pop)))
      break
    except:
      pass

print("Población creada!!")

Iniciando población...

Individuo 1 creado:
Individuo 1 evaluado.

Individuo 2 creado:
Individuo 2 evaluado.

Individuo 3 creado:
Individuo 3 evaluado.

Individuo 4 creado:
Individuo 4 evaluado.

Individuo 5 creado:
Individuo 5 evaluado.

Individuo 6 creado:
Individuo 6 evaluado.

Individuo 7 creado:
Individuo 7 evaluado.

Individuo 8 creado:
Individuo 8 evaluado.

Individuo 9 creado:
Individuo 9 evaluado.

Individuo 10 creado:
Individuo 10 evaluado.

Individuo 11 creado:
Individuo 11 evaluado.

Individuo 12 creado:
Individuo 12 evaluado.

Individuo 13 creado:
Individuo 13 evaluado.

Individuo 14 creado:
Individuo 14 evaluado.

Individuo 15 creado:
Individuo 15 evaluado.

Individuo 16 creado:
Individuo 16 evaluado.

Individuo 17 creado:
Individuo 17 creado:


/usr/local/lib/python3.7/dist-packages/torch/nn/init.py:403: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Individuo 17 evaluado.

Individuo 18 creado:
Individuo 18 evaluado.

Individuo 19 creado:
Individuo 19 evaluado.

Individuo 20 creado:
Individuo 20 creado:
Individuo 20 evaluado.

Población creada!!


In [7]:
#Mostrar aptitudes, precisión y num. parámetros de población inicial
for network in pop:
  print("f:{}, acc:{}, param:{}".format(network[1],network[2],network[3]))
max(pop, key = lambda x: x[1])

f:0.7088424666666666, acc:0.6000952380952381, param:74828
f:0.7820305333333333, acc:0.6897619047619048, param:5352
f:0.9614039666666666, acc:0.9826666666666667, param:176418
f:0.7870654666666667, acc:0.6970952380952381, param:6008
f:0.9469934999999999, acc:0.984, param:278710
f:0.9788176, acc:0.979, param:43216
f:0.8501251999999999, acc:0.7862857142857143, param:1832
f:0.9448379333333332, acc:0.9281904761904762, param:32636
f:0.9489969333333332, acc:0.9296190476190476, param:11576
f:0.9676161333333333, acc:0.9574761904761905, param:17448
f:0.9679473999999999, acc:0.9575714285714285, param:15684
f:0.9763871999999999, acc:0.969, param:12752
f:0.9673503666666667, acc:0.9575238095238096, param:19442
f:0.9291740666666666, acc:0.9123809523809524, param:63284
f:0.8213645666666667, acc:0.7465238095238095, param:8014
f:0.9614474666666666, acc:0.9462380952380952, param:6128
f:0.9337932333333332, acc:0.9064761904761904, param:4934
f:0.9368768333333333, acc:0.9127619047619048, param:13710
f:0.9154

[<EncodingClass.Encoding at 0x7f7d1b7be490>, 0.9788176, 0.979, 43216]

In [8]:
T = 10 #Number of generations
print("Iniciando genético...")
for t in range(T):
  print('Generation: ', t)
    
  #Parents Selection
  parents = []
  while len(parents) < int(N/2):
    #Tournament Selection
    tournament = random.sample(pop, t_size)
    p1 = selection(tournament, 'max')
    tournament = random.sample(pop, t_size)
    p2 = selection(tournament, 'max')
    while p1 == p2:
      tournament = random.sample(pop, t_size)
      p2 = selection(tournament, 'max')  
        
    parents.append(p1)
    parents.append(p2)
    
    #Reproduction
  offspring = []
  while len(offspring) < int(N/2):
    par = random.sample(parents, 2)
    #Crossover + Mutation
    if cr >= random.uniform(0,1): #Crossover
      p1 = par[0][0]
      p2 = par[1][0]
      c1, c2 = crossover(p1, p2)
            
    #Mutation
    if mr >= random.uniform(0,1):
      mutation(c1)
            
    if mr >= random.uniform(0,1):
      mutation(c2)

    #Try to evaluate the first offspring
    try:
      network1 = decoding(c1)
      cnn1 = CNN(c1, network1[0], network1[1], network1[2])
      acc1_list = []
      training('1', device1, cnn1, num_epochs, loss_func,train_dl, test_dl, lr, w, max_params, acc1_list)
      print("Descendiente {} evaluado.".format(len(offspring)+1))
      offspring.append([c1,acc1_list[0][1],acc1_list[0][2],acc1_list[0][3]])
    except:
      c1_random = random.sample(pop,1)[0]
      print("Descendiente {} aleatorio de pop.".format(len(offspring)+1))
      offspring.append([c1_random[0],c1_random[1],c1_random[2],c1_random[3]])

    #Try to evaluate the second offspring
    try:
      network2 = decoding(c2)
      cnn2 = CNN(c2, network2[0], network2[1], network2[2])
      acc2_list = []
      training('2', device1, cnn2, num_epochs, loss_func,train_dl, test_dl, lr, w, max_params, acc2_list)
      print("Descendiente {} evaluado.".format(len(offspring)+1))
      offspring.append([c2,acc2_list[0][1],acc2_list[0][2],acc2_list[0][3]])
    except:
      c2_random = random.sample(pop,1)[0]
      print("Descendiente {} aleatorio de pop.".format(len(offspring)+1))
      offspring.append([c2_random[0],c2_random[1],c2_random[2],c2_random[3]])

  #Replacement with elitism
  pop = pop + offspring
  pop.sort(reverse = True, key = lambda x: x[1])
  pop = pop[:N]
    
  leader = max(pop, key = lambda x: x[1])
  bestAcc.append(leader[2])
  bestF.append(leader[1])
  bestParams.append(leader[3])
    
        
  print('Best fitness: ', leader[1])
  print('Best accuracy: ', leader[2])
  print('Best No. of Params: ', leader[3])
  print('No. of Conv. Layers: ', leader[0].n_conv)
  print('No. of FC Layers: ', leader[0].n_full)
  print('--------------------------------------------')

print("Fin genético!!")

Iniciando genético...
Generation:  0
Descendiente 1 evaluado.
Descendiente 2 evaluado.
Descendiente 3 evaluado.
Descendiente 4 evaluado.
Descendiente 5 evaluado.
Descendiente 6 evaluado.
Descendiente 7 evaluado.
Descendiente 8 evaluado.
Descendiente 9 evaluado.
Descendiente 10 evaluado.
Best fitness:  0.9826973666666666
Best accuracy:  0.9839523809523809
Best No. of Params:  40462
No. of Conv. Layers:  3
No. of FC Layers:  1
--------------------------------------------
Generation:  1
Descendiente 1 evaluado.
Descendiente 2 evaluado.
Descendiente 3 evaluado.
Descendiente 4 evaluado.
Descendiente 5 evaluado.
Descendiente 6 evaluado.
Descendiente 7 evaluado.
Descendiente 8 evaluado.
Descendiente 9 evaluado.
Descendiente 10 evaluado.
Best fitness:  0.9826973666666666
Best accuracy:  0.9839523809523809
Best No. of Params:  40462
No. of Conv. Layers:  3
No. of FC Layers:  1
--------------------------------------------
Generation:  2
Descendiente 1 evaluado.
Descendiente 2 evaluado.
Descendie

/usr/local/lib/python3.7/dist-packages/torch/nn/init.py:403: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Descendiente 4 evaluado.
Descendiente 5 aleatorio de pop.
Descendiente 6 evaluado.
Descendiente 7 evaluado.
Descendiente 8 evaluado.
Descendiente 9 evaluado.
Descendiente 10 evaluado.
Best fitness:  0.9831764999999999
Best accuracy:  0.984
Best No. of Params:  37490
No. of Conv. Layers:  3
No. of FC Layers:  1
--------------------------------------------
Generation:  3
Descendiente 1 evaluado.
Descendiente 2 evaluado.
Descendiente 3 evaluado.
Descendiente 4 evaluado.
Descendiente 5 evaluado.
Descendiente 6 evaluado.
Descendiente 7 evaluado.
Descendiente 8 evaluado.
Descendiente 9 evaluado.
Descendiente 10 evaluado.
Best fitness:  0.9858397
Best accuracy:  0.9862857142857143
Best No. of Params:  30402
No. of Conv. Layers:  4
No. of FC Layers:  1
--------------------------------------------
Generation:  4
Descendiente 1 evaluado.
Descendiente 2 evaluado.
Descendiente 3 evaluado.
Descendiente 4 aleatorio de pop.
Descendiente 5 evaluado.
Descendiente 6 evaluado.
Descendiente 7 evaluado.
De

In [11]:
T = 10 #Number of generations
print("Iniciando genético...")
for t in range(T):
  print('Generation: ', t)
    
  #Parents Selection
  parents = []
  while len(parents) < int(N/2):
    #Tournament Selection
    tournament = random.sample(pop, t_size)
    p1 = selection(tournament, 'max')
    tournament = random.sample(pop, t_size)
    p2 = selection(tournament, 'max')
    while p1 == p2:
      tournament = random.sample(pop, t_size)
      p2 = selection(tournament, 'max')  
        
    parents.append(p1)
    parents.append(p2)
    
    #Reproduction
  offspring = []
  while len(offspring) < int(N/2):
    par = random.sample(parents, 2)
    #Crossover + Mutation
    if cr >= random.uniform(0,1): #Crossover
      p1 = par[0][0]
      p2 = par[1][0]
      c1, c2 = crossover(p1, p2)
            
    #Mutation
    if mr >= random.uniform(0,1):
      mutation(c1)
            
    if mr >= random.uniform(0,1):
      mutation(c2)

    #Try to evaluate the first offspring
    try:
      network1 = decoding(c1)
      cnn1 = CNN(c1, network1[0], network1[1], network1[2])
      acc1_list = []
      training('1', device1, cnn1, num_epochs, loss_func,train_dl, test_dl, lr, w, max_params, acc1_list)
      print("Descendiente {} evaluado.".format(len(offspring)+1))
      offspring.append([c1,acc1_list[0][1],acc1_list[0][2],acc1_list[0][3]])
    except:
      c1_random = random.sample(pop,1)[0]
      print("Descendiente {} aleatorio de pop.".format(len(offspring)+1))
      offspring.append([c1_random[0],c1_random[1],c1_random[2],c1_random[3]])

    #Try to evaluate the second offspring
    try:
      network2 = decoding(c2)
      cnn2 = CNN(c2, network2[0], network2[1], network2[2])
      acc2_list = []
      training('2', device1, cnn2, num_epochs, loss_func,train_dl, test_dl, lr, w, max_params, acc2_list)
      print("Descendiente {} evaluado.".format(len(offspring)+1))
      offspring.append([c2,acc2_list[0][1],acc2_list[0][2],acc2_list[0][3]])
    except:
      c2_random = random.sample(pop,1)[0]
      print("Descendiente {} aleatorio de pop.".format(len(offspring)+1))
      offspring.append([c2_random[0],c2_random[1],c2_random[2],c2_random[3]])

  #Replacement with elitism
  pop = pop + offspring
  pop.sort(reverse = True, key = lambda x: x[1])
  pop = pop[:N]
    
  leader = max(pop, key = lambda x: x[1])
  bestAcc.append(leader[2])
  bestF.append(leader[1])
  bestParams.append(leader[3])
    
        
  print('Best fitness: ', leader[1])
  print('Best accuracy: ', leader[2])
  print('Best No. of Params: ', leader[3])
  print('No. of Conv. Layers: ', leader[0].n_conv)
  print('No. of FC Layers: ', leader[0].n_full)
  print('--------------------------------------------')

print("Fin genético!!")

Iniciando genético...
Generation:  0
Descendiente 1 evaluado.
Descendiente 2 evaluado.
Descendiente 3 evaluado.
Descendiente 4 evaluado.
Descendiente 5 evaluado.
Descendiente 6 evaluado.
Descendiente 7 evaluado.
Descendiente 8 evaluado.
Descendiente 9 evaluado.
Descendiente 10 evaluado.
Best fitness:  0.9874343666666667
Best accuracy:  0.9878095238095238
Best No. of Params:  26882
No. of Conv. Layers:  4
No. of FC Layers:  2
--------------------------------------------
Generation:  1
Descendiente 1 evaluado.
Descendiente 2 evaluado.
Descendiente 3 evaluado.
Descendiente 4 evaluado.
Descendiente 5 evaluado.
Descendiente 6 aleatorio de pop.
Descendiente 7 evaluado.
Descendiente 8 evaluado.
Descendiente 9 evaluado.
Descendiente 10 evaluado.
Best fitness:  0.9874343666666667
Best accuracy:  0.9878095238095238
Best No. of Params:  26882
No. of Conv. Layers:  4
No. of FC Layers:  2
--------------------------------------------
Generation:  2
Descendiente 1 evaluado.
Descendiente 2 evaluado.
D

/usr/local/lib/python3.7/dist-packages/torch/nn/init.py:403: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Descendiente 4 evaluado.
Descendiente 5 aleatorio de pop.
Descendiente 6 evaluado.
Descendiente 7 aleatorio de pop.
Descendiente 8 evaluado.
Descendiente 9 evaluado.
Descendiente 10 evaluado.
Best fitness:  0.9874343666666667
Best accuracy:  0.9878095238095238
Best No. of Params:  26882
No. of Conv. Layers:  4
No. of FC Layers:  2
--------------------------------------------
Generation:  3
Descendiente 1 evaluado.
Descendiente 2 evaluado.
Descendiente 3 evaluado.
Descendiente 4 evaluado.
Descendiente 5 evaluado.
Descendiente 6 evaluado.
Descendiente 7 evaluado.
Descendiente 8 evaluado.
Descendiente 9 evaluado.
Descendiente 10 evaluado.
Best fitness:  0.9874343666666667
Best accuracy:  0.9878095238095238
Best No. of Params:  26882
No. of Conv. Layers:  4
No. of FC Layers:  2
--------------------------------------------
Generation:  4
Descendiente 1 evaluado.
Descendiente 2 evaluado.
Descendiente 3 evaluado.
Descendiente 4 evaluado.
Descendiente 5 evaluado.
Descendiente 6 evaluado.
Desce

In [12]:
#Mostrar aptitudes, precisión y num. parámetros de población final
for network in pop:
  print("f:{}, acc:{}, param:{}".format(network[1],network[2],network[3]))
max(pop, key = lambda x: x[1])

f:0.9874343666666667, acc:0.9878095238095238, param:26882
f:0.9873703333333333, acc:0.988904761904762, param:32420
f:0.9873703333333333, acc:0.988904761904762, param:32420
f:0.9873703333333333, acc:0.988904761904762, param:32420
f:0.9868186333333333, acc:0.9876190476190476, param:30098
f:0.9868186333333333, acc:0.9876190476190476, param:30098
f:0.9868186333333333, acc:0.9876190476190476, param:30098
f:0.9866676999999999, acc:0.9867142857142858, param:26882
f:0.9861397, acc:0.9867142857142858, param:30402
f:0.9861397, acc:0.9867142857142858, param:30402
f:0.9859397, acc:0.9864285714285714, param:30402
f:0.9859397, acc:0.9864285714285714, param:30402
f:0.9859397, acc:0.9864285714285714, param:30402
f:0.9859397, acc:0.9864285714285714, param:30402
f:0.9859157, acc:0.9872857142857143, param:34562
f:0.9859157, acc:0.9872857142857143, param:34562
f:0.9859157, acc:0.9872857142857143, param:34562
f:0.9859157, acc:0.9872857142857143, param:34562
f:0.9859015666666666, acc:0.9863809523809524, par

 0.9874343666666667,
 0.9878095238095238,
 26882]